In [81]:
import os
path = os.getcwd()
path = path.split('interactions', 1)[0] + 'interactions'
print(path)

/home/julia/Desktop/uni/enhancer-promoter-interactions


In [82]:
import pandas as pd

bed = pd.read_csv(path+'/results/whole_data/out_params_all_chrom.bedpe', delimiter='\t', header=None)
bed.columns = ['chr1', 'start1', 'end1', 'chr2', 'start2', 'end2', 'x', 'score', 'y', 'z', 'color']
print(bed.shape)
print(bed['chr1'].unique())
bed.head(10)


(1061804, 11)
['2L' '2R' '3L' '3R' '4' 'X']


,chr1,start1,end1,chr2,start2,end2,x,score,y,z,color
0,2L,10000910,10005105,2L,9950541,9951114,.,-0.004696,.,.,"255,0,0"
1,2L,10000910,10005105,2L,9951638,9952096,.,-0.005926,.,.,"255,0,0"
2,2L,10000910,10005105,2L,9953571,9954853,.,0.535730,.,.,"0,0,255"
3,2L,10000910,10005105,2L,9956462,9957908,.,0.569295,.,.,"0,0,255"
4,2L,10000910,10005105,2L,9963306,9963951,.,0.365345,.,.,"0,0,255"
5,2L,10000910,10005105,2L,9966422,9970768,.,0.552127,.,.,"0,0,255"
6,2L,10000910,10005105,2L,9972310,9973721,.,0.602123,.,.,"0,0,255"
7,2L,10000910,10005105,2L,9980748,9982050,.,0.022592,.,.,"0,0,255"
8,2L,10000910,10005105,2L,9982646,9985237,.,0.244502,.,.,"0,0,255"
9,2L,10000910,10005105,2L,9985950,9987615,.,-0.494903,.,.,"255,0,0"


In [83]:
bed['x'].replace('.', 0, inplace=True)
bed['y'].replace('.', 0, inplace=True)
bed['z'].replace('.', 0, inplace=True)
print(sum(bed['x']), sum(bed['y']), sum(bed['z']))
bed.drop(['x', 'y', 'z'], axis=1, inplace=True)
cicero_conns = bed.copy()
bed.head()

0 0 0


,chr1,start1,end1,chr2,start2,end2,score,color
0,2L,10000910,10005105,2L,9950541,9951114,-0.004696,"255,0,0"
1,2L,10000910,10005105,2L,9951638,9952096,-0.005926,"255,0,0"
2,2L,10000910,10005105,2L,9953571,9954853,0.535730,"0,0,255"
3,2L,10000910,10005105,2L,9956462,9957908,0.569295,"0,0,255"
4,2L,10000910,10005105,2L,9963306,9963951,0.365345,"0,0,255"


In [84]:
sum(cicero_conns['chr1'] != cicero_conns['chr2'])
cicero_conns = cicero_conns.drop('chr2', axis=1)
cicero_conns = cicero_conns.rename(columns={'chr1': 'chr'})
cicero_conns.head()

,chr,start1,end1,start2,end2,score,color
0,2L,10000910,10005105,9950541,9951114,-0.004696,"255,0,0"
1,2L,10000910,10005105,9951638,9952096,-0.005926,"255,0,0"
2,2L,10000910,10005105,9953571,9954853,0.535730,"0,0,255"
3,2L,10000910,10005105,9956462,9957908,0.569295,"0,0,255"
4,2L,10000910,10005105,9963306,9963951,0.365345,"0,0,255"


In [85]:
import pandas as pd

chromatin_loops = pd.read_excel(path+'/data/muszka/TableS2_Dmel_loops.xlsx')
chromatin_loops.head()

,loop ID,anchor ID,anchor chr,anchor start,anchor end,anchor summit,10-12h embryo neuron DHS count,loop size (in bp),anchor type,nearest TSS,anchor distance to TSS (in bp),meta-domain,loop clustering,loop type
0,L1,A1,chr2L,395670,395790,395730,1,16865113,intergenic,CG4213,6263,MD(L1-L3),clustered,meta-loop
1,L1,A19,chr2L,17260794,17260893,17260843,1,16865113,promoter,beat-IIIc,60,MD(L1-L3),clustered,meta-loop
2,L2,A1,chr2L,395670,395790,395730,1,16932336,intergenic,CG4213,6263,MD(L1-L3),clustered,meta-loop
3,L2,A20,chr2L,17328015,17328118,17328066,1,16932336,intergenic,CG45691,7303,MD(L1-L3),clustered,meta-loop
4,L3,A19,chr2L,17260794,17260893,17260843,1,67223,promoter,beat-IIIc,60,MD(L1-L3),clustered,intra-TAD loop


In [86]:
unique_col = ['loop ID', 'loop size (in bp)', 'loop clustering', 'loop type']
columns_to_duplicate = [col for col in list(chromatin_loops.columns) if col not in unique_col+['loop ID']]
print(columns_to_duplicate)
col_dict = {}
for col in columns_to_duplicate:
    col_dict[col] = ['first', 'last']
for col in unique_col:
    col_dict[col] = 'first'

['anchor ID', 'anchor chr', 'anchor start', 'anchor end', 'anchor summit', '10-12h embryo neuron DHS count', 'anchor type', 'nearest TSS', 'anchor distance to TSS (in bp)', 'meta-domain']


In [87]:
# join pairs of rows in chromatin_loops  with the same loop ID

print(chromatin_loops.shape)
chromatin_loops = chromatin_loops.groupby('loop ID').agg(col_dict)
chromatin_loops.head()


(136, 14)


anchor ID      anchor chr        anchor start           anchor end  \
            first last      first   last        first      last      first   
loop ID                                                                      
L1             A1  A19      chr2L  chr2L       395670  17260794     395790   
L10           A10  A13      chr2L  chr2L      6356962   9464540    6357076   
L11           A11  A12      chr2L  chr2L      6411271   9083037    6411420   
L12           A11  A14      chr2L  chr2L      6411271   9486317    6411420   
L13           A12  A14      chr2L  chr2L      9083037   9486317    9083252   

                  anchor summit            ...  nearest TSS             \
             last         first      last  ...        first       last   
loop ID                                    ...                           
L1       17260893        395730  17260843  ...       CG4213  beat-IIIc   
L10       9464794       6357019   9464720  ...       CG9500       numb   
L11       9083252       6411319   9083116  ...  DIP-epsilon     Toll-4   
L12       9486504       6411319   9486401  ...  DIP-epsilon        Gdi   
L13       9486504       9083116   9486401  ...       Toll-4        Gdi   

        anchor distance to TSS (in bp)         meta-domain               \
                                 first   last        first         last   
loop ID                                                                   
L1                                6263     60    MD(L1-L3)    MD(L1-L3)   
L10                               1241  14710  MD(L10,L12)  MD(L10,L12)   
L11                                  0    854      MD(L11)      MD(L11)   
L12                                  0   8291  MD(L10,L12)  MD(L10,L12)   
L13                                854   8291      MD(L13)      MD(L13)   

        loop ID loop size (in bp) loop clustering  loop type  
          first             first           first      first  
loop ID                                                       
L1           L1          16865113       clustered  meta-loop  
L10         L10           3107701       clustered  meta-loop  
L11         L11           2671797          single  meta-loop  
L12         L12           3075082       clustered  meta-loop  
L13         L13            403285          single  meta-loop  

[5 rows x 24 columns]

In [88]:
chromatin_loops.columns = chromatin_loops.columns.map('_'.join)
chromatin_loops.head()

,anchor ID_first,anchor ID_last,anchor chr_first,anchor chr_last,anchor start_first,anchor start_last,anchor end_first,anchor end_last,anchor summit_first,anchor summit_last,...,nearest TSS_first,nearest TSS_last,anchor distance to TSS (in bp)_first,anchor distance to TSS (in bp)_last,meta-domain_first,meta-domain_last,loop ID_first,loop size (in bp)_first,loop clustering_first,loop type_first
loop ID,,,,,,,,,,,,,,,,,,,,,
L1,A1,A19,chr2L,chr2L,395670,17260794,395790,17260893,395730,17260843,...,CG4213,beat-IIIc,6263,60,MD(L1-L3),MD(L1-L3),L1,16865113,clustered,meta-loop
L10,A10,A13,chr2L,chr2L,6356962,9464540,6357076,9464794,6357019,9464720,...,CG9500,numb,1241,14710,"MD(L10,L12)","MD(L10,L12)",L10,3107701,clustered,meta-loop
L11,A11,A12,chr2L,chr2L,6411271,9083037,6411420,9083252,6411319,9083116,...,DIP-epsilon,Toll-4,0,854,MD(L11),MD(L11),L11,2671797,single,meta-loop
L12,A11,A14,chr2L,chr2L,6411271,9486317,6411420,9486504,6411319,9486401,...,DIP-epsilon,Gdi,0,8291,"MD(L10,L12)","MD(L10,L12)",L12,3075082,clustered,meta-loop
L13,A12,A14,chr2L,chr2L,9083037,9486317,9083252,9486504,9083116,9486401,...,Toll-4,Gdi,854,8291,MD(L13),MD(L13),L13,403285,single,meta-loop


In [89]:
chromatin_loops.shape

(68, 24)

In [90]:
# cols anchor chr_first and anchor chr_last are identical
sum(chromatin_loops['anchor chr_first'] != chromatin_loops['anchor chr_last'])

0

In [91]:
chromatin_loops.drop(columns=['anchor chr_last'], inplace=True)
chromatin_loops.rename(columns={'anchor chr_first':'chr', 
                                'anchor start_first':'start1', 
                                'anchor start_last':'start2',
                                'anchor end_first':'end1',
                                'anchor end_last':'end2'}, inplace=True)
loop_conns = chromatin_loops[['chr', 'start1', 'end1', 'start2', 'end2']].copy()
loop_conns.head()

,chr,start1,end1,start2,end2
loop ID,,,,,
L1,chr2L,395670,395790,17260794,17260893
L10,chr2L,6356962,6357076,9464540,9464794
L11,chr2L,6411271,6411420,9083037,9083252
L12,chr2L,6411271,6411420,9486317,9486504
L13,chr2L,9083037,9083252,9486317,9486504


In [92]:
for col in ['start1', 'start2']:
    loop_conns.loc[:,'range_'+ col] = loop_conns.loc[:,col]+5000

for col in ['end1', 'end2']:
    loop_conns.loc[:,'range_' + col] = loop_conns.loc[:,col].apply(lambda x: max(0, x - 5000))
loop_conns['chr'] = loop_conns['chr'].apply(lambda x: x[3:])

loop_conns.head()

,chr,start1,end1,start2,end2,range_start1,range_start2,range_end1,range_end2
loop ID,,,,,,,,,
L1,2L,395670,395790,17260794,17260893,400670,17265794,390790,17255893
L10,2L,6356962,6357076,9464540,9464794,6361962,9469540,6352076,9459794
L11,2L,6411271,6411420,9083037,9083252,6416271,9088037,6406420,9078252
L12,2L,6411271,6411420,9486317,9486504,6416271,9491317,6406420,9481504
L13,2L,9083037,9083252,9486317,9486504,9088037,9491317,9078252,9481504


In [93]:
cicero_conns.head()
#change type to int
cicero_conns[['start1', 'start2', 'end1', 'end2']] = cicero_conns[['start1', 'start2', 'end1', 'end2']].astype(int)

In [94]:
def intersect_intervals(interval1, interval2):
    inter_start = max(interval1[0], interval2[0])
    inter_end = min(interval1[1], interval2[1])
    if inter_start <= inter_end:
        return [inter_start, inter_end]
    else:
        return False

In [95]:
cicero_conns['intersection'] = ['n']*cicero_conns.shape[0]
cicero_conns.reset_index(level=0, inplace=True)
cicero_conns.head() 

,index,chr,start1,end1,start2,end2,score,color,intersection
0,0,2L,10000910,10005105,9950541,9951114,-0.004696,"255,0,0",n
1,1,2L,10000910,10005105,9951638,9952096,-0.005926,"255,0,0",n
2,2,2L,10000910,10005105,9953571,9954853,0.535730,"0,0,255",n
3,3,2L,10000910,10005105,9956462,9957908,0.569295,"0,0,255",n
4,4,2L,10000910,10005105,9963306,9963951,0.365345,"0,0,255",n


In [97]:
cicero_2R = cicero_conns[cicero_conns['chr'] == '2R'].copy()

In [98]:
results = {'chr': [], 'intersection1': [], 
                                'intersection2': [],
                                'cicero1': [], 'cicero2': [], 
                                'loop1': [], 'loop2': [], 
                                'cicero_index': [], 'loop_index': []}
for chrom in set(cicero_2R['chr']).intersection(set(loop_conns['chr'])):
    cicero = cicero_conns[cicero_conns['chr'] == chrom]
    loops = loop_conns[loop_conns['chr'] == chrom]
    for (cicero1, cicero2, cicero_index) in zip(cicero[['start1', 'end1']].values, cicero[['start2', 'end2']].values, cicero['index'].values):
        for (loops1, loops2, loops_index) in zip(loops[['start1', 'end1']].values, loops[['start2', 'end2']].values, loops.index.values):
            if intersect_intervals(cicero1, loops1) and intersect_intervals(cicero2, loops2):
                results['chr'].append(chrom)
                results['intersection1'].append(intersect_intervals(cicero1, loops1))
                results['intersection2'].append(intersect_intervals(cicero2, loops2))
                results['cicero1'].append(cicero1)
                results['cicero2'].append(cicero2)
                results['loop1'].append(loops1)
                results['loop2'].append(loops2)
                results['cicero_index'].append(cicero_index)
                results['loop_index'].append(loops_index)
                cicero_conns.loc[cicero_index, 'intersection'] = 'y'
                print(cicero_index)
            else:
                if intersect_intervals(cicero1, loops2) and intersect_intervals(cicero2, loops1):
                    results['chr'].append(chrom)
                    results['intersection1'].append(intersect_intervals(cicero1, loops2))
                    results['intersection2'].append(intersect_intervals(cicero2, loops1))
                    results['cicero1'].append(cicero1)
                    results['cicero2'].append(cicero2)
                    results['loop1'].append(loops1)
                    results['loop2'].append(loops2)
                    results['cicero_index'].append(cicero_index)
                    results['loop_index'].append(loops_index)
                    cicero_conns.loc[cicero_index, 'intersection'] = 'y'

                    print(cicero_index)




In [77]:
result_df = pd.DataFrame(results)
result_df.to_csv(path+'/results/whole_data/intersected_cicero_loopssmall.csv', index=False, header=True)
cicero_conns.to_csv(path+'/results/whole_data/cicero_conns_intersected.csv', index=False, header=True)